### Hello, welcome to the FMR Python Automation tutorial!

We'll go over our code and how to use it.

**Structure**

Our code is based on just a couple of Python scripts. These are:
- `instrument_base.py`
- `bop50_8d.py`
- `hp_8673g.py`
- `srs_sr830.py`
- `fmr_experiment.py`

The first four files help us initialise and interact with our instruments. The last file `fmr_experiment.py` gives us some methods to then conduct various sweeps.

**What do I need to do to use these scripts?**

You'll need to make sure that whatever Python file you're using to do your analysis (be it a simple Python script or a Jupyter `.ipynb` notebook file) has those 5 files in the same directory. Of course you'll also want to make sure that the PC that you're running these files on is also connectied with our instruments via GPIB.

For instance, we're using this file, `tutorial.ipynb` to run some example code. So we'll want to make sure that those 5 Python scripts are in the same folder as this file. Let us begin.

### Calling our Experiment Class

Our experiment has a class, which helps it gather all the resources it needs among other things. If you're unfamiliar with this, don't worry. Simply import the class like so:

In [1]:
from fmr_experiment import Experiment

In [2]:
# Now let's initialise the class
E = Experiment()

Welcome to the FMR Experiment!
Here are some default experiment parameters.

PS Output Current (A) :	 6.29627
PS Output Voltage (V) :	 47.53
PS Output Mode (Current/Voltage) :	 Constant Current
SG Frequency :	 FR03000000000HZ
SG RF Output :	 On
SG RF Output Level :	 LE-70.0DM
LIA Time Constant :	 0.3
LIA Sensivity :	 0.0002
Sensivity Delay (s) :	 3
Read Repetitions :	 1
Read Repetition Delay :	 0
Repetition Averaging Function :	 <function mean at 0x000001CE31F4BF40>
Read Delay :	 0.02
From 0 Delay (s) :	 4
Log File :	 C:\Users\physlab\Desktop\FMR_Python_Automation\SpinLab_FMR_Automation\Experiment_Logs\FMR_log_2023-6-23_13-0-46.log


Great! We've iniialised an `Experiment` object and have named it `E`. Once initialised, a bunch of experiment parameters get printed out. Let's go over them:

- **PS Output Current (A)** shows the measured power supply current in Amperes. It's a bit out of wack because the set current is actually 0.
- **PS Output Voltage (V)** shows the measured power supply voltage in Volts. Same reason as the measurent current for why it isn't 0.
- **PS Output Mode (Current/Voltage)** shows whether the power supply is in constant current or constant voltage mode.
- **SG Frequency** is the RF output level from the signal generator.
- **SG RF Output** shows if the signal generator's RF output is on.
- **SG RF Output Level** shows the RF output's level in dB.
- **LIA Time Constant** shows the time constant for our lock-in amplifier in seconds.
- **LIA Sensivity** shows the absolute largest signal it can measure in the current sensivity setting. The sensivity is in Volts or Amperes depending on the measurement setting.
- **Sensivity Delay (s)** is the amount of time (in seconds) paused when the LIA sensivity is changed. This is because some time is required after changing the sensivity before stable reading can be achieved.
- **Read Repetitions** is the amount of readings taken per measurement of A and B. More readings give us a more stable, accurate result.
- **Read Repetition Delay** is the amount of time (in seconds) between reach read repitition.
- **Repetition Averaging Function** is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averagings than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **Read Delay** is the delay (in seconds) after the frequency or field is changed, before a measurement is taken.
- **From 0 Delay** is the delay (in seconds) after the frequency or field begins from 0 (occurs when initial parameters are being set), before a measurement is made.
- **Log File** is the path to the experiment log file which records every instrument communication. It's always a good idea to look at the log file for clues when things are acting up for unknown reasons.

These are just some of the experiment parameters which might be useful for you to take note of before your experiment. Of course, you can change most of these.

### Available Methods

Now that the experiment object is initialised, we can talk about the methods available. This will be easy, don't worry. We only have three methods to be aware of:

- `sweep_field`
- `sweep_frequency`
- `make2D`

Let's go over each method and see how to use it.

### `sweep_field`

As the name suggests, this method is used to conduct a magnetic field sweep at a particular output RF frequency. This method's arguments are:

- **frequency** *number*: Enter the frequency (in GHz) at which you want to conduct the magnetic field sweep.
- **fields** *number array*: Enter the magnetic field values (in Oe) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **livefig** *bool*: Would you like to show a live plot of the sweep as it happens? Pass along `True` or `False` (it's `True` by default).
- **savefig** *bool*: Would you like to save the sweep plot after it's done? Pass along `True` or `False` (it's `True` by default).
- **closefig** *bool*: Would you like to close the liveplot after it's done? Pass along `True` or `False` (it's `False` by default). There's usually no reason to automatically close the figure after the sweep is done, except if you're doing a lot of sweeps at the same time. In that case, closing the figures after the sweeps are done can save memory.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'freq_3.0_GHz_field_80-120_Oe_-3_dB.csv'` by default. If you'd like to add a tring before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_freq_3.0_GHz_field_80-120_Oe_-3_dB.csv'`.
- **sen** *float*: Want to start your sweep with a particular sensitivity? Put it in here. The default value is $0.0002$ V.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averagings than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **return_XY** *bool*: Do you want the function to return 2 numpy arrays with the *X* and *Y* values respectively? If so, pass along `True` or `False`. The default value is `False`.

### `sweep_frequency`

As the name suggests, this method is used to conduct a radio frequency sweep at a particular magnetic field. This method's arguments are:

- **field** *number*: Enter the field (in Oe) at which you want to conduct the frequency sweep.
- **frequencies** *number array*: Enter the frequency values (in GHz) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **livefig** *bool*: Would you like to show a live plot of the sweep as it happens? Pass along `True` or `False` (it's `True` by default).
- **savefig** *bool*: Would you like to save the sweep plot after it's done? Pass along `True` or `False` (it's `True` by default).
- **closefig** *bool*: Would you like to close the liveplot after it's done? Pass along `True` or `False` (it's `False` by default). There's usually no reason to automatically close the figure after the sweep is done, except if you're doing a lot of sweeps at the same time. In that case, closing the figures after the sweeps are done can save memory.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'field_100_Oe_freq_3-10_GHz_-3_dB.csv'` by default. If you'd like to add a tring before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_field_100_Oe_freq_3-10_GHz_-3_dB.csv'`.
- **sen** *float*: Want to start your sweep with a particular sensitivity? Put it in here. The default value is $0.0002$ V.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averagings than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **return_XY** *bool*: Do you want the function to return 2 numpy arrays with the *X* and *Y* values respectively? If so, pass along `True` or `False`. The default value is `False`.

### `make2D`

Would you like to make a 2D sweep of power absorption along the frequency and field axes? This method's arguments are:


- **frequencies** *number array*: Enter the frequency values (in GHz) through which you'd like to sweep in an array.
- **fields** *number array*: nter the magnetic field values (in Oe) through which you'd like to sweep in an array.
- **save_dir** *string*: Enter the path of the directory where you'd like to save your sweep data. If save directory does not exist, the program will create it (given that the parent directories exist).
- **primary** *string*: Whenever we conduct a 2D sweep, the sweep needs to select one of the parmeters (frequency or field) to be the 'primary' parameter with the other being the 'secondary' parameter. Pass along either `'frequency'` or `'field'` in this argument to decide the primary parameter. The defaault is `'frequency'`.
- **channel** *string*: both *X* and *Y* channel measurements from the LIA can show power absorption. Which channel do you want to use in the 2D sweep? Pass along `'X'` or `'Y'` in the argument here to decide. The default is `'X'`.
- **livefig** *bool*: Would you like to show a live plot of an individual sweep as it happens? Pass along `True` or `False` (it's `False` by default). Note that this is different from the 2D liveplot itself, which is always shown.
- **savefig** *bool*: Would you like to save each individual sweep plot after it's done? Pass along `True` or `False` (it's `False` by default). Note that this is different from the 2D liveplot itself, which is always saved.
- **closefig** *bool*: Would you like to close each individual sweep liveplot after it's done? Pass along `True` or `False` (it's `False` by default). If you're doing a lot of sweeps at the same time (which tends to happen during a 2D sweep), closing the figures after the sweeps are done can save memory. Note that this is different from the 2D liveplot itself.
- **file_prefix** *string*: The sweep's savefiles usually look something like `'2Dsweep_freq_3-10_GHz_field_100-500_Oe_-3_dB_channel_X_Unintegrated.npy'` by default. If you'd like to add a tring before this default fileneme, you can pass it along to the `file_prefix` argument. For example, I can pass along the string `'NiFe_'` here if I'm investigating a NiFe sample to get a filename `'NiFe_2Dsweep_freq_3-10_GHz_field_100-500_Oe_-3_dB_channel_X_Unintegrated.npy'`.
- **sen** *float*: Want to start your sweeps with a particular sensitivity? Put it in here. The default value is $0.0002$ V. Note that if the sensitivity changes during one individual sweep, it resets to this value before the next individual sweep.
- **sen_delay** *number*: How long (in seconds) do you want to wait before the LIA resumes taking readings after its sensitivity changes? By default this takes $3$ s. This delay is good to have as the LIA needs some time after sensitivity changes in order to stabilise.
- **read_reps** *int*: How many readings should the LIA take before making a measurement? This is $1$ by default. More repetitions usually help with unstable readings (especially at higher frequencies).
- **rep_delay** *number*: How long (in seconds) do you want to wait in between LIA readings? The default value is $0$ s.
- **read_delay** *number*: How long (in seconds) do you want to wait after the frequency or field is changed, before a measurement is taken? The default value is $0$ s.
- **from0delay** *number*: How long (in seconds) do you want to wait after the frequency or field begins from $0$ (occurs when initial parameters are being set), before a measurement is made? The default value is $4$ s.
- **avg_func** *function*: This is the function which creates a measurement from our repeated readings. By default, this is a simple mean function (`np.mean`). However, sometimes, you want to do more complex averagings than a simple mean. For instance, the experiment class also has a method called `avg_mid_50` which takes in an array as argument, and returns the mean of the middle $50^{th}$ percentile of array elements. This can help stabilise erratic measurements, especially at higher frequencies.
- **integrate** *bool*: Do you want the *X* or *Y* channel sweep to be integrated? If so, pass along `True` or `False`. The default value is `False`. Note that this doesn't work very well for unstable power absorption values.